In [16]:
import numpy as np
import cv2 as cv
import glob

In [17]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

In [18]:
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('FishEyeTestset/*.jpg')
images

['FishEyeTestset/left05.jpg',
 'FishEyeTestset/right09.jpg',
 'FishEyeTestset/left04.jpg',
 'FishEyeTestset/right01.jpg',
 'FishEyeTestset/right13.jpg',
 'FishEyeTestset/right07.jpg',
 'FishEyeTestset/left14.jpg',
 'FishEyeTestset/left08.jpg',
 'FishEyeTestset/left13.jpg',
 'FishEyeTestset/left06.jpg',
 'FishEyeTestset/left12.jpg',
 'FishEyeTestset/right11.jpg',
 'FishEyeTestset/left02.jpg',
 'FishEyeTestset/right03.jpg',
 'FishEyeTestset/right06.jpg',
 'FishEyeTestset/right08.jpg',
 'FishEyeTestset/left03.jpg',
 'FishEyeTestset/left07.jpg',
 'FishEyeTestset/right05.jpg',
 'FishEyeTestset/right12.jpg',
 'FishEyeTestset/left09.jpg',
 'FishEyeTestset/right14.jpg',
 'FishEyeTestset/right02.jpg',
 'FishEyeTestset/left01.jpg',
 'FishEyeTestset/right04.jpg',
 'FishEyeTestset/left11.jpg']

In [19]:
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7,6), None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        cv.drawChessboardCorners(img, (7,6), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)

cv.destroyAllWindows()

In [20]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [21]:
img = cv.imread('FishEyeTestset/left12.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [22]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult1.png', dst)

True

In [23]:
# undistort
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult2.png', dst)

True

In [25]:
cv.imwrite('no calibration.png', img)

True